### Метаданные

In [7]:
import os
os.environ["OPENAI_API_KEY"] = "sk-kFANI4tptQfVJAZKpbRxDNBVODFDrOdk"
os.environ["NEO4J_URI"] ="bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "kodin"
os.environ["NEO4J_PASSWORD"] = "12345678"

In [8]:
import os
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter

from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.graphs import Neo4jGraph

### Подключаемся к графу

In [9]:
graph = Neo4jGraph(database='testdb')

### Индексируем Header_node

In [10]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(base_url='https://api.proxyapi.ru/openai/v1'),
    search_type="hybrid",
    node_label="Header_node",
    text_node_properties=["text"],
    embedding_node_property="embedding",
    database='testdb'
)


In [12]:
query = "Как скачать "
docs_with_score_1 = vector_index.similarity_search_with_score(query, k=5)

In [13]:
docs_with_score_1

[(Document(page_content='\ntext: ПП поставляется в электронном виде и включает в себя следующий набор файлов:'),
  1.0),
 (Document(page_content='\ntext: Наличие доступа в Интернет:'),
  0.992750232111142),
 (Document(page_content='\ntext: Наличие доступа в Интернет'),
  0.9921729612176229),
 (Document(page_content='\ntext: В окне настроек необходимо выбрать один из режимов проверки (по умолчанию - медленный):'),
  0.9900717030523256),
 (Document(page_content='\ntext: В главном окне отображаются вкладки с информацией по модулям:'),
  0.9889705307251003)]

## demo retrive

In [43]:
def print_neighbors(conn, node_id, radius=1):
    query = f"""
    MATCH (n)-[*1..{radius}]-(m)
    WHERE properties(n).id = "{node_id}"
    RETURN collect(distinct m) as neighbors
    """
    results = conn.query(query)
    neighbors = results[0]['neighbors'] if results else []
    
    print(f"Neighbors of node with ID {node_id} (radius {radius}):")
    for neighbor in neighbors:
        try:
            print(f"Node ID: {neighbor.id}, Properties: {dict(neighbor)}")
        except:
            pass

In [46]:
for doc, score in docs_with_score_1:
    print("|" * 80)
    print("Score: ", score)
    print("=" * 80)
    name = doc.page_content.replace('\ntext: ','')
    print('doc.page_content: ', doc.page_content)
    print("=" * 80)
    
    resp = graph.query(f"""
                        MATCH (n)-[*1..2]->(m)
                        WHERE properties(m).id = "{name}"
                        RETURN n
                        """)
    
    resp2 = graph.query(f"""
                        MATCH (n)-[*1..5]->(m)
                        WHERE properties(n).id = "{name}"
                        RETURN m
                        """)
    
    print('-')  
    for node in resp2[::-1]:
        node_to = node['m']['text'].strip()
        node_to = " ".join(node_to.split(' '))
        
        print(f"{name}  - > {node['m']['text']}")
        
    
    for node in resp:
        
        node_from = node['n']['text'].strip()
        node_from = " ".join(node_from.split())
        
        print(f"{node_from}  - > {name}")
    print('-')

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Score:  1.0
doc.page_content:  
text: ПП поставляется в электронном виде и включает в себя следующий набор файлов:
-
ПП поставляется в электронном виде и включает в себя следующий набор файлов:  - > •	Dozor.1.0.0.rpm (для Alt Linux) или Dozor.1.0.0.deb (для Astra Linux) содержит файлы установки.
•	Руководство пользователя (данный документ).

2.1 Назначение и структура  - > ПП поставляется в электронном виде и включает в себя следующий набор файлов:
2 Назначение и основные возможности ПП  - > ПП поставляется в электронном виде и включает в себя следующий набор файлов:
content  - > ПП поставляется в электронном виде и включает в себя следующий набор файлов:
content  - > ПП поставляется в электронном виде и включает в себя следующий набор файлов:
-
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Score:  0.992750232111142
doc.page_content:  
text: Наличие доступа в Интернет:
-
